In [49]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from re import search

%matplotlib inline

In [2]:
pip install -U selenium

Requirement already up-to-date: selenium in c:\programdata\anaconda3\lib\site-packages (3.141.0)
Note: you may need to restart the kernel to use updated packages.


In [3]:
import sys
print(sys.version)

3.7.4 (default, Aug  9 2019, 18:34:13) [MSC v.1915 64 bit (AMD64)]


In [5]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
from http.cookiejar import CookieJar
import urllib
import urllib.request
import html.parser
import requests
from requests.exceptions import HTTPError
from socket import error as SocketError
from selenium import webdriver
from xml.dom.minidom import parseString
import json        
import xmlrpc.client

base_url="https://www.bigbasket.com/product/get-products"
cat_path="/?slug=fruits-vegetables"
page="&page="
qry_prm="&tab_type=[%22all%22]&sorted_on=popularity&listtype=pc"

page_vals = ["1","2","3","4","5","6","7","8","9"]

tlcs  = [] #tlc_s
llcn  = [] #llc_n
brand = [] #p_brand
mrp   = [] #mrp
sp    = [] #sp
disc  = [] #dis_val
sku   = [] #sku
desc  = [] #p_desc
purl  = [] #absolute_url

def append (myjson, index, key):
    if key == "tlc_s": 
        tlcs.append(myjson[index])
    elif key == "llc_n":    
        llcn.append(myjson[index])
    elif key == "p_brand":    
        brand.append(myjson[index])
    elif key == "mrp":    
        mrp.append(myjson[index])
    elif key == "sp":    
        sp.append(myjson[index])
    elif key == "dis_val":    
        disc.append(myjson[index])
    elif key == "sku":    
        sku.append(myjson[index])
    elif key == "p_desc":    
        desc.append(myjson[index])
    elif key == "absolute_url":    
        purl.append(myjson[index])

def get_all(myjson, key):
    if type(myjson) == str:
        myjson = json.loads(myjson)
    if type(myjson) is dict:
        for jsonkey in myjson:
            if jsonkey == key:
                append (myjson, jsonkey, key)
            elif type(myjson[jsonkey]) in (list, dict):
                get_all(myjson[jsonkey], key)
    elif type(myjson) is list:
        for item in myjson:
            if item == key:
                append (myjson, item, key)
            elif type(item) in (list, dict):
                get_all(item, key)
    
for page_index in page_vals: 
    req_url = base_url+cat_path+page+page_index+qry_prm
    print (req_url)
    try:
        req=urllib.request.Request(req_url)
        cj = CookieJar()
        opener = urllib.request.build_opener(urllib.request.HTTPCookieProcessor(cj))
        req.add_header('User-Agent','Mozilla/5.0')
        response = opener.open(req)
        p_json = response.read().decode('utf8', errors='ignore')
        json_object = get_all (p_json, "tlc_s")
        json_object = get_all (p_json, "llc_n")
        json_object = get_all (p_json, "p_brand")
        json_object = get_all (p_json, "mrp")
        json_object = get_all (p_json, "sp")
        json_object = get_all (p_json, "dis_val")
        json_object = get_all (p_json, "sku")
        json_object = get_all (p_json, "p_desc")
        json_object = get_all (p_json, "absolute_url")
        response.close()
    except urllib.request.HTTPError as inst:
        output = format(inst)
        print(output)
    
print ("tlcs", len(tlcs))
print ("llcn", len(llcn))
print ("brand", len(brand))
print ("mrp", len(mrp))
print ("sp", len(sp))
print ("disc", len(disc))
print ("sku", len(sku))
print ("desc", len(desc))
print ("purl", len(purl))


df = pd.DataFrame()
df["product"] = pd.Series(tlcs)
df["category"] = pd.Series(llcn)
df["brand"] = pd.Series(brand)
df["mrp"] = pd.Series(mrp)
df["sp"] = pd.Series(sp)
df["discount"] = pd.Series(disc)
df["sku"] = pd.Series(sku)
df["description"] = pd.Series(desc)
df["url"] = pd.Series(purl)

df.drop (df[df['product'] != 'fruits-vegetables'].index, inplace=True)
df.info()
df.to_csv('BigBasket.csv')

https://www.bigbasket.com/product/get-products/?slug=fruits-vegetables&page=1&tab_type=[%22all%22]&sorted_on=popularity&listtype=pc
https://www.bigbasket.com/product/get-products/?slug=fruits-vegetables&page=2&tab_type=[%22all%22]&sorted_on=popularity&listtype=pc
https://www.bigbasket.com/product/get-products/?slug=fruits-vegetables&page=3&tab_type=[%22all%22]&sorted_on=popularity&listtype=pc
https://www.bigbasket.com/product/get-products/?slug=fruits-vegetables&page=4&tab_type=[%22all%22]&sorted_on=popularity&listtype=pc
https://www.bigbasket.com/product/get-products/?slug=fruits-vegetables&page=5&tab_type=[%22all%22]&sorted_on=popularity&listtype=pc
https://www.bigbasket.com/product/get-products/?slug=fruits-vegetables&page=6&tab_type=[%22all%22]&sorted_on=popularity&listtype=pc
https://www.bigbasket.com/product/get-products/?slug=fruits-vegetables&page=7&tab_type=[%22all%22]&sorted_on=popularity&listtype=pc
https://www.bigbasket.com/product/get-products/?slug=fruits-vegetables&page=

### DATA CLEANING

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 208 entries, 3 to 279
Data columns (total 9 columns):
product        208 non-null object
category       208 non-null object
brand          208 non-null object
mrp            208 non-null object
sp             208 non-null object
discount       208 non-null object
sku            208 non-null int64
description    208 non-null object
url            208 non-null object
dtypes: int64(1), object(8)
memory usage: 16.2+ KB


In [7]:
df.columns

Index(['product', 'category', 'brand', 'mrp', 'sp', 'discount', 'sku',
       'description', 'url'],
      dtype='object')

In [8]:
df.head()

,product,category,brand,mrp,sp,discount,sku,description,url
3,fruits-vegetables,"Potato, Onion & Tomato",Fresho,25.00,20.00,20,10000148,Onion,/pd/10000148/fresho-onion-1-kg/
4,fruits-vegetables,"Potato, Onion & Tomato",Fresho,46.25,37.00,20,10000159,Potato,/pd/10000159/fresho-potato-1-kg/
9,fruits-vegetables,Mangoes,bb Combo,333.75,267.00,20,1200345,Mango - Banganpalli,/pd/1200345/bb-combo-mango-banganpalli-3-kg/
10,fruits-vegetables,Mangoes,Fresho,111.25,89.00,20,10000298,Banganapalli Mango,/pd/10000298/fresho-banganapalli-mango-1-kg/
11,fruits-vegetables,Mangoes,Fresho,111.25,89.00,20,10000298,Banganapalli Mango,/pd/10000298/fresho-banganapalli-mango-1-kg/


In [9]:
#Dropping columns that dont mean much for us here. Dropping sku
df.drop('sku', axis=1, inplace=True)


In [10]:
df.head(3)

,product,category,brand,mrp,sp,discount,description,url
3,fruits-vegetables,"Potato, Onion & Tomato",Fresho,25.00,20.00,20,Onion,/pd/10000148/fresho-onion-1-kg/
4,fruits-vegetables,"Potato, Onion & Tomato",Fresho,46.25,37.00,20,Potato,/pd/10000159/fresho-potato-1-kg/
9,fruits-vegetables,Mangoes,bb Combo,333.75,267.00,20,Mango - Banganpalli,/pd/1200345/bb-combo-mango-banganpalli-3-kg/


In [11]:
df.columns

Index(['product', 'category', 'brand', 'mrp', 'sp', 'discount', 'description',
       'url'],
      dtype='object')

In [12]:
df.columns=['Product', 'Category', 'Brand', 'MRP', 'ActualPrice', 'Discount', 'ProductDesc','Url']

In [21]:
#From Url column deriving Product Quantity
df=df[~df.Url.str.contains("combo", na=False)]

In [51]:
def getQtforPcs(qt):
    array=qt.split('x')
    array = [int(i) for i in array] 
    return np.prod(array)

In [52]:
def getQuantity(x):
    array = x.split('/')
    qttemp=array[3].split('-')
    size=len(qttemp)
    wt=qttemp[size-1]
    qt=qttemp[size-2]
    if(wt=='kg' or wt=='gm' or wt=='g' or wt=='pcs' or wt=='pc'):
            if(search('x', qt)):
                qt=getQtforPcs(qt)
            if(wt=='g'):
                return qt+'gm'
            elif(wt=='pcs'):
                return qt+'pc'
            else:
                return qt+wt
    elif(wt=='bunch'):
        return qttemp[size-3]+qttemp[size-2]
    
df['Quantity'] = df['Url'].apply(lambda x: getQuantity(x))

TypeError: ufunc 'add' did not contain a loop with signature matching types dtype('<U11') dtype('<U11') dtype('<U11')

In [35]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 202 entries, 3 to 279
Data columns (total 9 columns):
Product        202 non-null object
Category       202 non-null object
Brand          202 non-null object
MRP            202 non-null object
ActualPrice    202 non-null object
Discount       202 non-null object
ProductDesc    202 non-null object
Url            202 non-null object
Quantity       202 non-null object
dtypes: object(9)
memory usage: 15.8+ KB


In [24]:
print(df.isnull().sum())

Product        0
Category       0
Brand          0
MRP            0
ActualPrice    0
Discount       0
ProductDesc    0
Url            0
Quantity       0
dtype: int64


In [36]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(df['Quantity'])

3          1kg
4          1kg
10         1kg
11         1kg
77       500gm
78         1pc
79         1pc
80         1pc
81         1kg
82         1kg
83       500gm
84       500gm
85         1pc
86         2pc
87       500gm
88       250gm
89       250gm
90         4pc
91       100gm
92       100gm
93       500gm
94       250gm
95       250gm
96         1kg
97       100gm
98       500gm
99       250gm
100      500gm
101      500gm
102        4pc
103        1kg
104        1pc
105        1pc
106      500gm
107        1kg
108        1kg
109      100gm
110        1kg
111        1kg
112      500gm
114      500gm
115      100gm
116      2x4pc
117      3x2pc
118        2pc
119        4pc
120        4pc
121      250gm
122      500gm
123        3pc
124      500gm
125      500gm
126      100gm
128      100gm
129        1kg
130      500gm
131      250gm
132      250gm
133        4pc
134        6pc
135        6pc
137        1kg
138      500gm
139        4pc
140      100gm
141      250gm
142      2

In [ ]:
#Dropping whenever quantity is a combo. As we cant compare combos accross website and provide recommendation.
#Combos can be of any variety, itll be too complex to compare combos.

In [ ]:
#df=df[~df.Url.str.contains("combo", na=False)]

In [ ]:
df.info()

In [37]:
str1='5x3'
array=str1.split('x')

In [38]:
array

['5', '3']

In [42]:
import numpy as np
mylist = [ 2, 3, 4, 6] 
result = np.prod(np.array(mylist))  
result

144